<a href="https://colab.research.google.com/github/kyle-woodward/EEpythonNotebooks/blob/main/Export_Model_Inference_Pools_to_Fire_BurnProb_California_FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pools to Fire BurnProb California FCNN
## Use a deployed AI platform EEified model to create and export inference images
## downloads and reads in the sample_.json and modelname.json dictionaries to automatically setup feature stack and AI model for inference

## Setting up the environment

### Importing packages and checking versions

codecarbon only works after installing and then reseting the notebook. 

In [ ]:
#!pip install codecarbon

Figure out which region you are in with the command below. Training requires many reads from GCP. If your bucket is located halfway across the world from your instance do a Factory Resart and check again. 

In [ ]:
!curl ipinfo.io

{
  "ip": "34.148.229.157",
  "hostname": "157.229.148.34.bc.googleusercontent.com",
  "city": "North Charleston",
  "region": "South Carolina",
  "country": "US",
  "loc": "32.8546,-79.9748",
  "org": "AS15169 Google LLC",
  "postal": "29415",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
import os
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()
# from codecarbon import EmissionsTracker

# tracker = EmissionsTracker()
# tracker.start()
# get numpy and matplotlib.pyplot
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=zxO7ks-UDdjE8714dLwv4He6YgO3mCLmFPGl01zl88c&tc=T_mWuvFSIN6HH-MlMVxvvcNW4srqI8bWTlJS4eQuk5s&cc=pzvSN7SODHDNEy0tIHQ9lDnJDUGiynXJrsdCxcDOrTU

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh83UhHiXF82z6uuo4lN0cKDQDkshXmoX9II2XvZKnakm7L4Yq7znE

Successfully saved authorization token.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K

print(tf.__version__)

2.8.0


In [ ]:
# Folium setup.
# import folium
# print(folium.__version__)

## Load sample and model .json files

In [ ]:
import json
model_name = 'vgg16unet_model_kdw_sigmoid_mae_v18_rev2' # without a file extension
vers = 'v18_rev2'

# load sample .json made from SampleGeneration/ModelTraining workflow
sample_json= os.popen(f'gsutil ls gs://landfire/lucas/pools2fire_tf/sample_{vers}*.json').read().split('\n')[:-1][-1] # download latest sample version .json if there's more than one of same version
#print(sample_json)
local = sample_json.split('/')[-1]
!gsutil cp $sample_json $local
with open(local) as f:
  model_dict = json.load(f)
print('tf model info',model_dict)
bands = model_dict.get('bands')
response = model_dict.get('response')
features = model_dict.get('features')
kernal_size = model_dict.get('kernal_size')

# load ai model .json file made from EEificationDeployment workflow
os.popen(f"gsutil cp gs://landfire/lucas/pools2fire_tf/{model_name}.json {model_name}.json").read()
with open(f'{model_name}.json') as f:
  ai_model_dict = json.load(f)
estimator = ai_model_dict.get('estimator')
version = ai_model_dict.get('version')
project = ai_model_dict.get('project')
region = ai_model_dict.get('region')
output_dict = ai_model_dict.get('output')
print('eeified model info',ai_model_dict)

Copying gs://landfire/lucas/pools2fire_tf/sample_v18_rev2_1651770421.json...
/ [1 files][  862.0 B/  862.0 B]                                                
Operation completed over 1 objects/862.0 B.                                      
tf model info {'uid': 'sample_v18_rev2_1651770421', 'training': 'training_patches_v18_rev2', 'testing': 'testing_patches_v18_rev2', 'validation': 'val_patches_v18_rev2', 'bands': ['elevation', 'slope', 'trasp', 'TMP_p10', 'TMP_median', 'TMP_p90', 'WIND_p10', 'WIND_median', 'WIND_p90', 'SPFH_p10', 'SPFH_median', 'SPFH_p90', 'WDIR_p10', 'WDIR_median', 'WDIR_p90', 'ACPC01_p10', 'ACPC01_median', 'ACPC01_p90'], 'response': ['BP'], 'features': ['elevation', 'slope', 'trasp', 'TMP_p10', 'TMP_median', 'TMP_p90', 'WIND_p10', 'WIND_median', 'WIND_p90', 'SPFH_p10', 'SPFH_median', 'SPFH_p90', 'WDIR_p10', 'WDIR_median', 'WDIR_p90', 'ACPC01_p10', 'ACPC01_median', 'ACPC01_p90', 'BP'], 'kernal_size': 256, 'categorical': [], 'categorical_dict': {}, 'n': 20, 'N': 300,

### stack all possible input features we've ever used and then pass json's 'bands' values to image.select()

In [ ]:
def trasp(img):
  #(1 - cos(aspect-30))/2
  aspect = ee.Image(img)
  trasp = (ee.Image.constant(1).subtract(aspect.subtract(30).cos())).divide(2)
  return trasp.rename('trasp')

srtm = ee.Image('USGS/SRTMGL1_003')
ag_fast = ee.Image('projects/pyregence-ee/assets/lucas/conus/Aboveground-Fast_2022_02').rename('agfast').float()
ag_med = ee.Image('projects/pyregence-ee/assets/lucas/conus/Aboveground-Medium_2022_02').rename('agmed').float()
ag_slo = ee.Image('projects/pyregence-ee/assets/lucas/conus/Aboveground-Slow_2022_02').rename('agslo').float()
ag_vfast = ee.Image('projects/pyregence-ee/assets/lucas/conus/Aboveground-Very-Fast_2022_02').rename('agvfast').float()
bg_fast = ee.Image('projects/pyregence-ee/assets/lucas/conus/Belowground-Fast_2022_02').rename('bgfast').float()
bg_slo = ee.Image('projects/pyregence-ee/assets/lucas/conus/Belowground-Slow_2022_02').rename('bgslo').float()
bg_vfast = ee.Image('projects/pyregence-ee/assets/lucas/conus/Belowground-Very-Fast_2022_02').rename('bgvfast').float()
c_root = ee.Image('projects/pyregence-ee/assets/lucas/conus/Coarse-Roots_2022_02').rename('croot').float()
f_root = ee.Image('projects/pyregence-ee/assets/lucas/conus/Fine-Roots_2022_02').rename('froot').float()
fol = ee.Image('projects/pyregence-ee/assets/lucas/conus/Foliage_2022_02').rename('foliage').float()
merch = ee.Image('projects/pyregence-ee/assets/lucas/conus/Merchantable_2022_02').rename('merch').float()
otherw = ee.Image('projects/pyregence-ee/assets/lucas/conus/Other-Wood_2022_02').rename('otherw').float()
snbran = ee.Image('projects/pyregence-ee/assets/lucas/conus/Snag-Branch_2022_02').rename('snbran').float()
snstem = ee.Image('projects/pyregence-ee/assets/lucas/conus/Snag-Stem_2022_02').rename('snstem').float()
#last_dist = ee.Image('projects/pyregence-ee/assets/lucas/conus/Recent-Disturbance-Masked_2022_02').rename('ldist').float()
last_dist = ee.Image('projects/pyregence-ee/assets/lucas/conus/Recent-Disturbance_2022_02').rename('ldist').float()
elevation = srtm.select('elevation').float()
slope = ee.Terrain.slope(elevation).float()
aspect = ee.Terrain.aspect(elevation).float()
trasp = trasp(aspect)
stateclass = ee.Image('projects/pyregence-ee/assets/lucas/conus/stateclass_onlyforest').rename('sclass')
evt = ee.Image('projects/pyregence-ee/assets/conus/landfire/evt/LF2016_EVT_200').rename('evt')
age = ee.Image('projects/pyregence-ee/assets/lucas/conus/Age-Class_2022_02').rename('age')
pyromes = ee.Image("projects/pyregence-ee/assets/conus/usfs/pyromes").rename('pyrome')

# combining carbon pools
live_ag = merch.add(otherw).add(fol).rename('live_ag')
dead_ag = snstem.add(snbran).add(ag_fast).add(ag_med).add(ag_slo).add(ag_vfast).rename('dead_ag')
live_bg = c_root.add(f_root).rename('live_bg')
dead_bg = bg_fast.add(bg_slo).add(bg_vfast).rename('dead_bg')

ag_all = ag_fast.add(ag_med).add(ag_slo).add(ag_vfast).rename('ag_all')
bg_all = bg_fast.add(bg_slo).add(bg_vfast).rename('bg_all')

# weather
# convert from specific to relative humidity first?
spfh_p10 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_SPFH_2011_2021").select('SPFH_p10').float()#.unitScale(0, 0.015077958814799786)
spfh_median = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_SPFH_2011_2021").select('SPFH_median').float()#.unitScale(0, 0.018254483118653297)
spfh_p90 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_SPFH_2011_2021").select('SPFH_p90').float()#.unitScale(0, 0.02191421203315258)

# convert from C to F first? 
tmp_p10 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_TMP_2011_2021").select('TMP_p10').float()#.unitScale(-24.222455978393555, 25.068578720092773)
tmp_median = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_TMP_2011_2021").select('TMP_median').float()#.unitScale(-6.363852500915527,	27.317960739135742)
tmp_p90 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_TMP_2011_2021").select('TMP_p90').float()#.unitScale(0, 42.87779235839844)

wind_p10 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_WIND_2011_2021").select('WIND_p10').float()#.unitScale(0, 3.938079833984375)
wind_median = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_WIND_2011_2021").select('WIND_median').float()#.unitScale(0, 9.37088680267334)
wind_p90 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_WIND_2011_2021").select('WIND_p90').float()#.unitScale(0, 17.119548797607422)

# better scaling method than min/max compress? i.e. cosine transform like aspect TRASP equation?
wdir_p10 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_WDIR_2011_2021").select('WDIR_p10').float()#.unitScale(0, 268.53846153846155)
wdir_median = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_WDIR_2011_2021").select('WDIR_median').float()#.unitScale(0, 332.5821167883212)
wdir_p90 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_WDIR_2011_2021").select('WDIR_p90').float()#.unitScale(0, 354.5677758318739)

# convert from mm/kg^2 to mm first?
precip_p10 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_ACPC01_2011_2021").select('ACPC01_p10').float()#.unitScale(0, 0.09782296419143677)
precip_median = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_ACPC01_2011_2021").select('ACPC01_median').float()#.unitScale(0, 0.1984066665172577)
precip_p90 = ee.Image("projects/pyregence-ee/assets/conus/weather/RTMA_ACPC01_2011_2021").select('ACPC01_p90').float()#.unitScale(0, 16.100494384765625)


wx = tmp_p10.addBands(tmp_median).addBands(tmp_p90)\
            .addBands(wind_p10).addBands(wind_median).addBands(wind_p90)\
            .addBands(spfh_p10).addBands(spfh_median).addBands(spfh_p90)\
            .addBands(wdir_p10).addBands(wdir_median).addBands(wdir_p90)\
            .addBands(precip_p10).addBands(precip_median).addBands(precip_p90)



all_stack = ag_fast.addBands(ag_med).addBands(ag_slo).addBands(ag_vfast)\
                  .addBands(bg_fast).addBands(bg_slo).addBands(bg_vfast)\
                  .addBands(c_root).addBands(f_root).addBands(fol).addBands(merch)\
                  .addBands(otherw).addBands(snbran).addBands(snstem)\
                  .addBands(last_dist).addBands(elevation).addBands(slope).addBands(aspect).addBands(trasp)\
                  .addBands(stateclass).addBands(evt).addBands(age)\
                  .addBands(live_ag).addBands(dead_ag).addBands(live_bg).addBands(dead_bg)\
                  .addBands(ag_all).addBands(bg_all)\
                  .addBands(pyromes).addBands(wx).unmask()
                  

print(all_stack.select(bands).bandNames().getInfo())

['elevation', 'slope', 'trasp', 'TMP_p10', 'TMP_median', 'TMP_p90', 'WIND_p10', 'WIND_median', 'WIND_p90', 'SPFH_p10', 'SPFH_median', 'SPFH_p90', 'WDIR_p10', 'WDIR_median', 'WDIR_p90', 'ACPC01_p10', 'ACPC01_median', 'ACPC01_p90']


In [ ]:
print(estimator)
print(version)
print(project)
print(region)
print(output_dict)
# v1649682345

vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR
v1651845684
pyregence-ee
us-central1
pools2BP


In [ ]:
array_image = all_stack.select(bands).toArray()
print(array_image.bandNames().getInfo())
input_depth = all_stack.select(bands).bandNames().size()
#print(input_depth.getInfo())
 # Point to the model hosted on AI Platform.  If you specified a region other
 # than the default (us-central1) at model creation, specify it here.

MODEL_NAME= estimator
VERSION_NAME= version
PROJECT= project
model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME,
    version=VERSION_NAME,
    #set up model in us-east1, deafult is us-central1
    region=region,
     # Can be anything, but don't make it too big.
     #model trained using 256 array
     #tile size needs to be factor of 256 e.g. (4,8,16,32,64,128,256)
    inputTileSize=[32, 32],
     #if wanting an input overlap the final tile size needs to match the factor
     #final tilesize =  tilesize + overlap*2
     #e.g. 32 = 24 + (4 * 2)
    inputOverlapSize= [32, 32],
     # Keep this the same as your training data.
    proj=ee.Projection('EPSG:4326').atScale(30),
    fixInputProj=True,
    # set input shape to depth of input (# of bands in input stack)
    inputShapes= {'array':[input_depth]},
     # Note the names here need to match what you specified in the
     # output dictionary you passed to the EEifier.
    outputBands={output_dict: {
        'type': ee.PixelType.float(),
         #use dimesions if multiple predictions are outputed. 
         #'dimensions': 1
      }
    },
)


 # model.predictImage outputs a one dimensional array image that
 # packs the output nodes of your model into an array.  These
 # are class probabilities that you need to unpack into a 
 # multiband image with arrayFlatten().  If you want class
 # labels, use arrayArgmax() as follows.
predictions = model.predictImage(array_image)#//.rename('BP_predicted')
print(predictions.bandNames().getInfo())
# print(model)

['array']
['pools2BP']


In [ ]:
grassValley_geom = ee.Image("projects/pyregence-ee/assets/lucas/tensorflow/BPpredictions_linear_huber_GrassValley").geometry()
CA = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME','California')).first().geometry()
bboxes = ee.FeatureCollection("projects/pyregence-ee/assets/lucas/tensorflow/inference_bboxes")
ids = bboxes.aggregate_array('system:index').getInfo()
#print(ids)
# for id in ids:
#   id_ = id[-1]
#   geom = bboxes.filter(ee.Filter.eq('system:index',id)).first()
#   #print(id_)
task = ee.batch.Export.image.toAsset(
  image=predictions,
  assetId= 'projects/pyregence-ee/assets/lucas/tensorflow/' + MODEL_NAME + '_grassValley',
  description=MODEL_NAME + '_grassValley',
  region=grassValley_geom,
  scale=30,
  maxPixels=1e12
  )

task.start()
print('exporting prediction to bbox ')

exporting prediction to bbox 
